# 03 — Insights & Sanity Checks

- How much data was missing and got imputed?
- Did imputation materially change distributions/seasonality?
- Simple predictive sanity: do lagged usages correlate with current usage?

In [ ]:
import pandas as pd, numpy as np

csv_path = "../data/data_test.csv"
df = pd.read_csv(csv_path)
miss_rate = df['lusage'].isna().mean()
n = len(df)
print(f"Missing lusage: {miss_rate:.1%} ({df['lusage'].isna().sum()} of {n})")

In [ ]:
# After you've run one of the imputation notebooks or scripts and saved output to CSV,
# load it here and compare. For demo, reuse baseline calculation quickly:
dfb = df.copy()
med1 = dfb.groupby(['year','month','mozip'])['lusage'].median()
med2 = dfb.groupby(['zipcode','month'])['lusage'].median()
global_med = dfb['lusage'].median()
def fill_row(r):
    if pd.notnull(r['lusage']): return r['lusage']
    v = med1.get((r['year'],r['month'],r['mozip']), np.nan)
    if pd.isna(v): v = med2.get((r['zipcode'],r['month']), np.nan)
    if pd.isna(v): v = global_med
    return v
dfb['lusage_imp'] = dfb.apply(fill_row, axis=1)

pre_avg = df['lusage'].mean()
post_avg = dfb['lusage_imp'].mean()
print("Pre avg:", pre_avg, "Post avg (baseline):", post_avg)